# Construyendo una red neuronal con PyTorch

## Librerías

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from sklearn.metrics import classification_report
from tqdm.notebook import tqdm

## Cargando los datos del CIFAR10

- El conjunto de datos a utilizar es el **[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)**.
- Es un conjunto estándar para hacer *reconocimiento de imágenes*.
- Buscamos entrenar un clasificador que reconozca que está siendo mostrado en la imagen de un conjunto fijo de categorías posibles.
- El CIFAR-10 está compuesto por imágenes a color de 32x32 píxeles representadas como tensores de 32x32x3, donde la tercera dimensión representa el *channel* (i.e. el color en RGB). Los valores representan la intensidad de cada color en dicho pixel.
- La salida son 10 clases: avión, auto, pájaro, gato, siervo, perro, sapo, caballo, bote, camión.
- La librería `torchvision` nos facilita obtener el conjunto de datos.

In [ ]:
CIFAR_CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 
                 'dog', 'frog', 'horse', 'ship', 'truck')
BATCH_SIZE = 128  # For mini-batch gradient descent
EPOCHS = 2

# This is to normalize from PILImage to Torch Tensors in range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

## Explorando el CIFAR-10

Vamos a explorar algunas de las imágenes que muestra el conjunto de datos del CIFAR-10.

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # desnormalizar
    npimg = img.numpy()
    plt.imshow(numpy.transpose(npimg, (1, 2, 0)))
    plt.show()

# obtener algunas imágenes de entrenamiento
dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images[:5]
labels = labels[:5]

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print('\t'.join('%5s' % CIFAR_CLASSES[labels[j]] for j in range(5)))

## Contruyendo la red neuronal

- Comenzaremos por construir un *perceptrón multicapa* que es la red neuronal más común.
- La capa más básica es la *lineal* (i.e. la completamente conectada) que se define en [`torch.nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html).
    - Internamente tiene dos tensores: una matriz de pesos y un vector de biases. PyTorch nos abstrae de eso.
    - Es equivalente a la capa [`Dense`](https://keras.io/api/layers/core_layers/dense/) en [Keras](https://keras.io/).

### Modelo Secuencial

El mayor tipo de abstracción lo provee [`torch.nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html). Es un modelo donde cada capa se aplica después de la siguiente. Es limitado en sus usos posibles, pero tiene la ventaja de abstraer completamente del funcionamiento. Es equivalente al modelo [`tf.keras.Sequential`](https://keras.io/api/models/sequential/#sequential-class) de Keras.

In [ ]:
model = nn.Sequential(
    nn.Linear(32 * 32 * 3, 512),
    nn.ReLU(),
    nn.Linear(512, 10)
)

Podemos inspeccionar el modelo simplemente imprimiéndolo

In [ ]:
print(model)

Podemos ver que nos devuelve el modelo inicializado al azar:

In [ ]:
img_check, lbl_check = dataiter.next()
img_check, lbl_check = img_check[0], lbl_check[0]
img_check = img_check.view(-1)  # Transformar la imagen en un vector, es equivalente a img_check.view(32*32*3)

print(f"Clase real: {CIFAR_CLASSES[lbl_check]}")
print(f"Clase devuelta: {CIFAR_CLASSES[model(img_check).argmax().item()]}")

### Modelo de PyTorch

Es el modelo base de PyTorch que permite un grado de personalización mucho más profundo. 

Los modelos en PyTorch heredan de la clase [`torch.nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module). La clase posee dos métodos que requieren definirse:
1. `__init__`: Define la estructura de la red (i.e las capas que tiene).
2. `forward`: Define como interactúan las capas de la red (i.e. las operaciones).

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_layer1 = nn.Linear(32 * 32 * 3, 512)
        self.hidden_layer2 = nn.Linear(512, 256)
        self.output_layer = nn.Linear(256, 10)
    
    def forward(self, x: torch.Tensor):
        x = self.hidden_layer1(x)  # Go through hidden layer
        x = F.relu(x)  # Activation Function
        x = self.hidden_layer2(x)
        x = F.relu(x)
        x = self.output_layer(x)  # Output Layer
        return x

model = MLP()

Podemos inspeccionar el modelo simplemente imprimiéndolo

In [ ]:
print(model)

Podemos ver que nos devuelve el modelo inicializado al azar (notar que este modelo no requiere reordenar la matriz que representa la imagen):

In [ ]:
img_check, lbl_check = dataiter.next()
img_check, lbl_check = img_check[0], lbl_check[0]

print(f"Clase real: {CIFAR_CLASSES[lbl_check]}")
print(f"Clase devuelta: {CIFAR_CLASSES[model(img_check.view(-1)).argmax().item()]}")

# Hiperparámetros

## Funciones de Activación

- Una red neuronal con activación lineal no tiene mucho más poder de representación que un algoritmo lineal.
- Para expresar no linearidad en la red neuronal se necesitan funciones no lineales de activación.
- Una función de activación común es la *sigmoide* (o logística).
- PyTorch soporta varias funciones de activación: rectified linea unit (ReLU), tangenge hiperbólica, sigmoide "dura", etc.
    - Hoy en día, por sus propiedades, ReLU suele ser la más utilizada [1].
- La función de activación *Softmax* suele utilizarse al final de una red de multiples clases, tiene como objetivo transformar un vector de *scores* en un vector probabilístico.
    - Si bien solía ser muy común en las primeras versiones de muchos frameworks de deep learning, con el tiempo se dejó de utilizar y se incluye directamente en la función de costo de manera transparente al usuario.
- En PyTorch la mayoría de estas funciones se definen en el módulo [`torch.nn.functional`](https://pytorch.org/docs/stable/nn.functional.html).

<img style="margin:auto;width:100%;" src="images/activation_functions.png" alt="Funciones de activación" title="Funciones de activación"/>
<div style="text-align:right;font-size:0.75em;">Fuente: <a href="https://ujjwalkarn.me/2016/08/09/quick-intro-neural-networks/" target="_blank">https://ujjwalkarn.me/2016/08/09/quick-intro-neural-networks/</a></div>

In [ ]:
model = nn.Sequential(
    nn.Linear(32 * 32 * 3, 512),
    nn.Sigmoid(),
    nn.Linear(512, 10),
    nn.Softmax(-1)
)

print(model(img_check.view(-1)))

## Preparando el modelo para entrenarlo

- Para minimizar una red neuronal necesitamos *calcular sus gradientes*.
    - Esto se hace con el algoritmo de *retropropagación*.
- PyTorch tiene la capacidad de hacerlo automáticamente.
    - Esto se conoce como _diferenciación automática_ y es algo común en los frameworks de deep learning.
- Necesitaremos definir dos puntos para entrenar un modelo: la función de costo y el algoritmo de optimización.

### Función de costo

- La función de costo puede cambiar de acuerdo al tipo de problema (clasificación binaria/multiclase o regresión).
    - La funciones más comunes son la media del error cuadrático (_mean squared error_) para regresión y la entropía cruzada (_crossentropy_) para clasificación.
    - Las funciones de costo necesitan ser diferenciables, las más comunes ya están implementadas (con sus respectivas derivadas) en PyTorch.

### Algoritmo de optimización

- El algoritmo de optimización es el que entrena la red. Existen varios, que en si son variaciones del algoritmo de _descenso por la gradiente_.
    - El módulo [`torch.optim`](https://pytorch.org/docs/stable/optim.html) tiene implementados varios algoritmos de optimización muy utilizados en aprendizaje automático.

<div style="text-align: center; margin: 5px 0;">
    <div style="display: inline-block;">
        <img src="images/contours_evaluation_optimizers.gif" alt="Optimización" style="width:350px;"/>
    </div>
    <div style="display: inline-block;">
        <img src="images/saddle_point_evaluation_optimizers.gif" alt="Optimización" style="width:350px;"/>
    </div>
</div>
<div style="text-align:right;font-size:0.75em;">Fuente: <a href="http://ruder.io/optimizing-gradient-descent/" target="_blank">http://ruder.io/optimizing-gradient-descent/</a></div>

In [ ]:
model = MLP()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Entrenamiento

- Ya teniendo todos los parámetros definidos, el modelo está listo para entrenarse.
- PyTorch no provee una solución "out-of-the-box" para el bucle de entrenamiento (o en inglés *training loop*), a diferencia de scikit-learn y Keras que proveen `fit` y `predict`. Existen algunas librerías que proveen dicha abstracción, pero dejamos como ejercicio [buscar alguna](https://pytorch.org/ecosystem/) si lo consideran necesario.
- En general si bien el hecho de tener que hacer el loop de entrenamiento desde cero es un poco tedioso, da lugar a mucha más personalización y control del proceso de entrenamiento.

In [ ]:
model.train()
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    running_loss = 0.0
    pbar = tqdm(trainloader)
    for i, data in enumerate(pbar, 1):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.view(inputs.shape[0], -1))
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i > 0 and i % 50 == 0:    # print every 50 mini-batches
            pbar.set_description(f"[{epoch + 1}, {i}] loss: {running_loss / 50:.4g}")
            running_loss = 0.0

### Guardar el modelo

Guardamos los parámetros del modelo entrenado:

In [ ]:
torch.save(model.state_dict(), "./data/cifar-model.pth")

## Evaluación

### Evaluación Manual

Podemos ver como funciona el modelo luego de un ciclo de entrenamiento:

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()
images, labels = images[:5], labels[:5]

# print images
imshow(torchvision.utils.make_grid(images.view(images.shape[0], -1)))
print('GroundTruth: ', ' '.join('%5s' % CIFAR_CLASSES[labels[j]] for j in range(5)))

Cargamos los pesos del modelo:

In [ ]:
model = MLP()
model.load_state_dict(torch.load("./data/cifar-model.pth"))
model.eval();  # Activate evaluation mode

Vemos como está funcionando

In [ ]:
outputs = model(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % CIFAR_CLASSES[predicted[j]] for j in range(5)))

### Evaluación General

Si queremos medir que tan bien está funcionando en general, necesitamos correr para todos los valores de test:

In [ ]:
y_true = []
y_pred = []
with torch.no_grad():
    for data in tqdm(testloader):
        inputs, labels = data
        outputs = model(inputs.view(inputs.shape[0], -1))
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

print(classification_report(y_true, y_pred, target_names=CIFAR_CLASSES))

## Entrenamiento en GPU

El entrenamiento en GPU requiere que "enviemos" el modelo y los datos al GPU. Para ello, se deben cambiar algunas cosas.

### Definir el dispositivo

Primero se define el dispositivo a utilizar. Si se dispone de GPU lo utilizamos, en caso contrario se selecciona el CPU.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Una vez seleccionado el dispositivo, debemos crear y enviar el modelo a dicho dispositivo.

In [ ]:
model = MLP()
model.to(device)
model.train();

De aqui los pasos son prácticamente los mismos que para el [entrenamiento](#Entrenamiento). Es importante que definamos nuevamente el algoritmo de optimización sobre los parámetros nuevos que en este caso se enviaron al GPU. Luego, durante el *training loop* debemos asegurarnos de, previo a pasar los datos al modelo, estos sean cargados en el dispositivo:

In [ ]:
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(tqdm(trainloader), 1):
        inputs = inputs.to(device)
        labels = labels.to(device)
        # run the rest of the algorithm as usual

## Regularización de la red

### Regularización de los pesos

- La red puede regularizarse penalizando los pesos.
- Los pesos se regularizan mediante alguna norma:
    - L1 es la suma del valor absoluto: ${\displaystyle \lambda \sum_{i=1}^{k} |w_i|}$
    - L2 es la suma del valor cuadrado, es la más común: ${\displaystyle \lambda \sum_{i=1}^{k} w_i^2}$
    - Elastic net es una combinación de ambas: ${\displaystyle \lambda_1 \sum_{i=1}^{k} |w_i| + \lambda_2 \sum_{i=1}^{k} w_i^2}$
- Para un análisis detallado de la diferencia entre L1 y L2 revisar [\[2\]](http://www.chioka.in/differences-between-l1-and-l2-as-loss-function-and-regularization/)

Varios (sino todos) de los algoritmos de optimización implementados en `torch.optim` vienen con una implementación de la norma L2 para regularización (que suele ser la norma por defecto), simplemente se debe seleccionar el valor de $\lambda$ que viene representado por el parámetro `weight_decay`.

In [ ]:
model = MLP()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), 
                       lr=0.001,
                       weight_decay=0.001)

Si queremos mayor control (e.g. implementar una versión de L1 o ElasticNet), debemos hacerlo como parte del *training loop*:

In [ ]:
l1_lambda = 0.001
model = model.to(device)
model.train()
for inputs, labels in tqdm(trainloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs.view(inputs.shape[0], -1))
    loss = loss_function(outputs, labels)

    l1_reg = torch.tensor(0.).to(device)
    for param in model.parameters():
        l1_reg += torch.norm(param, p=1)
    loss += l1_reg * l1_lambda

    loss.backward()
    optimizer.step()

El caso anterior regulariza todos los pesos, i.e. los pesos de las capas y los bias. Si sólo queremos regularizar los pesos, se puede utilizar [`nn.Module.named_parameters`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.named_parameters) y filtrar aquellos parámetros que tengan `bias` en el nombre.

In [ ]:
l1_reg = torch.tensor(0.)
for param_name, param_weight in model.named_parameters():
    if 'bias' not in param_name:
        l1_reg += torch.norm(param, p=1)

### Dropout

- Otra forma muy usada a la hora de regularizar es el **dropout** [3].
- Es extremadamente efectivo y simple.
- Es complementario a L1/L2/ElasticNet.
- Durante el entrenamiento se implementa apagando un neurón con alguna probabilidad **_p_** (un hiperparámetro).

<img style="margin:auto;width:75%;" src="images/dropout.jpeg" alt="Dropout" title="Dropout"/>
<div style="text-align:right;font-size:0.75em;">Fuente: Trabajo de Srivastava et al. [3]</div>

### Dropout en PyTorch

- Se aplica agregando capas al modelo.
- Se llaman capas [`torch.nn.Dropout`](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html) y se agrega a cada capa que se quiere regularizar.

In [ ]:
model = nn.Sequential(
    nn.Linear(32 * 32 * 3, 512),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(512, 10)
)

### Batch Normalization

- En general, para acelerar la convergencia de la red, se normalizan los features de entrada, de manera que todos estén en un rango similar.
- Esta idea también puede llevarse a las capas ocultas de la red.
- La idea de la "Normalización por Lotes" (*Batch Normalization*) [4] es reducir el rango en el que se mueven los valores de las neuronas ocultas.
- La manera en que se hace esto es restarle, a cada salida de cada capa oculta, la media del lote (batch) de datos de entrenamiento y dividirlo por la desviación estándar (a grandes razgos).
- Como resultado, la red converge más rápido e incluso se genera un efecto de regularización.

### Batch Normalization en PyTorch

- Se aplica agregando capas al modelo.
- Se llaman capas [`BatchNorm*`](https://pytorch.org/docs/stable/nn.html#normalization-layers), donde `*` se reemplaza por las dimensiones de entrada (`1d`, `2d`, `3d`) y se agrega a cada capa que se quiere normalizar.
- El `momentum` es un parámetro que decide cuánta información de los lotes anteriores se tiene en cuenta a la hora de normalizar el lote actual (en el trabajo original de *Batch Normalization*, este es de `0`).

In [ ]:
model = nn.Sequential(
    nn.Linear(32 * 32 * 3, 512),
    nn.BatchNorm1d(512, momentum=0.1),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(512, 10)
)

## Referencias

- [1] LeCun, Yann, Bengio, Yoshua, and Hinton, Geoffrey. "Deep learning." Nature 521, no. 7553 (2015): 436-444.
- [2] "Differences between L1 and L2 as Loss Function and Regularization". http://www.chioka.in/differences-between-l1-and-l2-as-loss-function-and-regularization/
- [3] Srivastava, Nitish, Geoffrey E. Hinton, Alex Krizhevsky, Ilya Sutskever, and Ruslan Salakhutdinov. "Dropout: a simple way to prevent neural networks from overfitting." Journal of machine learning research 15, no. 1 (2014): 1929-1958. Harvard.
- [4] Ioffe, S., & Szegedy, C. (2015). Batch normalization: Accelerating deep network training by reducing internal covariate shift. arXiv preprint arXiv:1502.03167.